# Implementation of a simple GAN using a 2 layered neural net.

In [37]:
# Importing TensorFlow

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import *
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [47]:
# Hyperparameters

batch = 32
X_dim = 784
Z_dim = 64
h_dim = 128
alpha = 0.001

In [48]:
def plot(samples):
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig
    

In [49]:
# Function to initialise weight matrix of a given shape
def weight_initialise(shape):
    value = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(value)



In [50]:
# Defining variables for the Discriminator net.

X = tf.placeholder(tf.float32, shape = [None,X_dim])

#Layer 1
D_Weights1 = weight_initialise([X_dim,h_dim])
D_Bias1 = tf.Variable(tf.zeros(shape=[h_dim]))

#Layer 2
D_Weights2 = weight_initialise([h_dim,1])
D_Bias2 = tf.Variable(tf.zeros(shape=[1]))


theta_D = [D_Weights1, D_Weights2, D_Bias1, D_Bias2]

In [51]:
# Similarly, for the Generator...

z = tf.placeholder(tf.float32,shape=[None,Z_dim])

#Layer 1
G_Weights1 = weight_initialise([Z_dim,h_dim])
G_Bias1 = tf.Variable(tf.zeros(shape=[h_dim]))

#Layer 2
G_Weights2 = weight_initialise([h_dim,X_dim])
G_Bias2 = tf.Variable(tf.zeros(shape=[X_dim]))


theta_G = [D_Weights1, D_Weights2, D_Bias1, D_Bias2]

In [52]:
# Defining the generator function

def generator(z):
    G_Activation1 = tf.nn.relu(tf.matmul(z,G_Weights1)+G_Bias1)
    G_Activation2 = tf.matmul(G_Activation1,G_Weights2) + G_Bias2
    generated = tf.nn.sigmoid(G_Activation2)
    return generated


# Defining the discriminator

def discriminator(x):
    D_Activation1 = tf.nn.relu(tf.matmul(x,D_Weights1)+D_Bias1)
    D_Activation2 = tf.matmul(D_Activation1,D_Weights2) + D_Bias2
    return D_Activation2
    

In [53]:
# Function to create initial z samples that are fed to generator

def sample_z(m,n):
    return np.random.uniform(-1., 1., size=[m, n])

In [54]:
G_sample = generator(z)

D_Real  = discriminator(X)

D_Fake  = discriminator(G_sample)

D_Target = 1./ batch

G_Target = 1./ (batch*2)

Z = tf.reduce_sum(tf.exp(-D_Real)) + tf.reduce_sum(tf.exp(-D_Fake))

D_loss = tf.reduce_sum(D_Target * D_Real) + tf.log(Z + 0.000000001)
G_loss = tf.reduce_sum(G_Target * D_Real) + tf.reduce_sum(G_Target * D_Fake) + tf.log(Z + 0.000000001)

D_solver = (tf.train.AdamOptimizer(learning_rate=alpha)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=alpha)
            .minimize(G_loss, var_list=theta_G))

sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [ ]:
# Training
if not os.path.exists('out/'):
    os.makedirs('out/')
i = 0
for it in range(1000000):
    X_mb, _ = mnist.train.next_batch(batch)
    z_mb = sample_z(batch, Z_dim)

    _, D_loss_curr = sess.run(
        [D_solver, D_loss], feed_dict={X: X_mb, z: z_mb}
    )

    _, G_loss_curr = sess.run(
        [G_solver, G_loss], feed_dict={X: X_mb, z: z_mb}
    )

    if it % 1000 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))
    if it % 100000 == 0:
        samples = sess.run(G_sample,feed_dict={z: sample_z(16, Z_dim)})
        
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)),bbox_inches='tight')
        i+=1
        plt.close(fig)

Iter: 0; D_loss: 4.313; G_loss: 4.362
Iter: 1000; D_loss: 4.289; G_loss: 4.342
Iter: 2000; D_loss: 4.567; G_loss: 4.519
Iter: 3000; D_loss: 4.881; G_loss: 4.656
Iter: 4000; D_loss: 4.935; G_loss: 4.782
Iter: 5000; D_loss: 4.613; G_loss: 4.458
Iter: 6000; D_loss: 4.502; G_loss: 4.484
Iter: 7000; D_loss: 4.417; G_loss: 4.426
Iter: 8000; D_loss: 4.331; G_loss: 4.418
Iter: 9000; D_loss: 4.446; G_loss: 4.354
Iter: 10000; D_loss: 4.227; G_loss: 4.379
Iter: 11000; D_loss: 4.189; G_loss: 4.363
Iter: 12000; D_loss: 4.246; G_loss: 4.326
Iter: 13000; D_loss: 4.125; G_loss: 4.339
Iter: 14000; D_loss: 4.126; G_loss: 4.365
Iter: 15000; D_loss: 4.225; G_loss: 4.389
Iter: 16000; D_loss: 4.185; G_loss: 4.359
Iter: 17000; D_loss: 4.238; G_loss: 4.409
Iter: 18000; D_loss: 4.301; G_loss: 4.408
Iter: 19000; D_loss: 4.337; G_loss: 4.464
Iter: 20000; D_loss: 4.418; G_loss: 4.531
Iter: 21000; D_loss: 4.573; G_loss: 4.607
Iter: 22000; D_loss: 4.604; G_loss: 4.572
Iter: 23000; D_loss: 4.679; G_loss: 4.64
Iter: 